In [9]:
import pandas as pd
from fbprophet import Prophet

In [10]:
df = pd.read_csv('calendar_borough.csv', error_bad_lines = False)
df

,Unnamed: 0,date,borough_id,room_typeid,price
0,0,2017-01-01,1,1,133.285714
1,1,2017-01-01,1,2,67.000000
2,2,2017-01-01,1,4,130.000000
3,3,2017-01-01,2,1,219.255682
4,4,2017-01-01,2,2,115.936170
...,...,...,...,...,...
31007,31007,2021-12-31,4,3,188.428571
31008,31008,2021-12-31,4,4,83.301370
31009,31009,2021-12-31,5,1,183.301587
31010,31010,2021-12-31,5,2,72.560000


In [11]:
df=df[['date', 'borough_id', 'room_typeid', 'price']]
df

,date,borough_id,room_typeid,price
0,2017-01-01,1,1,133.285714
1,2017-01-01,1,2,67.000000
2,2017-01-01,1,4,130.000000
3,2017-01-01,2,1,219.255682
4,2017-01-01,2,2,115.936170
...,...,...,...,...
31007,2021-12-31,4,3,188.428571
31008,2021-12-31,4,4,83.301370
31009,2021-12-31,5,1,183.301587
31010,2021-12-31,5,2,72.560000


In [12]:
df['combined'] = df.borough_id.astype(str) + df.room_typeid.astype(str)

In [13]:
df = df.rename(columns={"date": "ds", "price": "y"})

In [14]:
df=df[['ds', 'combined', 'y']]
df

,ds,combined,y
0,2017-01-01,11,133.285714
1,2017-01-01,12,67.000000
2,2017-01-01,14,130.000000
3,2017-01-01,21,219.255682
4,2017-01-01,22,115.936170
...,...,...,...
31007,2021-12-31,43,188.428571
31008,2021-12-31,44,83.301370
31009,2021-12-31,51,183.301587
31010,2021-12-31,52,72.560000


In [16]:
grouped = df.groupby('combined')
final = pd.DataFrame()
for g in grouped.groups:
    group = grouped.get_group(g)
    m = Prophet()
    m.fit(group)
    future = m.make_future_dataframe(periods=730)
    forecast = m.predict(future)    
    forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
    final = pd.merge(final, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [17]:
final 

,trend_x,yhat_lower_x,yhat_upper_x,trend_lower_x,trend_upper_x,additive_terms_x,additive_terms_lower_x,additive_terms_upper_x,weekly_x,weekly_lower_x,weekly_upper_x,yearly_x,yearly_lower_x,yearly_upper_x,multiplicative_terms_x,multiplicative_terms_lower_x,multiplicative_terms_upper_x,yhat_11,trend_y,yhat_lower_y,yhat_upper_y,trend_lower_y,trend_upper_y,additive_terms_y,additive_terms_lower_y,additive_terms_upper_y,weekly_y,weekly_lower_y,weekly_upper_y,yearly_y,yearly_lower_y,yearly_upper_y,multiplicative_terms_y,multiplicative_terms_lower_y,multiplicative_terms_upper_y,yhat_12,trend_x,yhat_lower_x,yhat_upper_x,trend_lower_x,...,multiplicative_terms_y,multiplicative_terms_lower_y,multiplicative_terms_upper_y,yhat_51,trend_x,yhat_lower_x,yhat_upper_x,trend_lower_x,trend_upper_x,additive_terms_x,additive_terms_lower_x,additive_terms_upper_x,weekly_x,weekly_lower_x,weekly_upper_x,yearly_x,yearly_lower_x,yearly_upper_x,multiplicative_terms_x,multiplicative_terms_lower_x,multiplicative_terms_upper_x,yhat_52,trend_y,yhat_lower_y,yhat_upper_y,trend_lower_y,trend_upper_y,additive_terms_y,additive_terms_lower_y,additive_terms_upper_y,weekly_y,weekly_lower_y,weekly_upper_y,yearly_y,yearly_lower_y,yearly_upper_y,multiplicative_terms_y,multiplicative_terms_lower_y,multiplicative_terms_upper_y,yhat_54
ds,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,103.442259,100.500708,116.711459,103.442259,103.442259,5.116056,5.116056,5.116056,-1.199493,-1.199493,-1.199493,6.315549,6.315549,6.315549,0.0,0.0,0.0,108.558315,56.097877,55.890805,59.793519,56.097877,56.097877,1.668145,1.668145,1.668145,-0.446501,-0.446501,-0.446501,2.114646,2.114646,2.114646,0.0,0.0,0.0,57.766023,142.380326,121.060259,139.356117,142.380326,...,0.0,0.0,0.0,139.406519,67.186607,64.519823,71.469458,67.186607,67.186607,0.731752,0.731752,0.731752,-0.591987,-0.591987,-0.591987,1.323739,1.323739,1.323739,0.0,0.0,0.0,67.918359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,103.475662,99.596635,116.178689,103.475662,103.475662,4.205070,4.205070,4.205070,-1.371440,-1.371440,-1.371440,5.576510,5.576510,5.576510,0.0,0.0,0.0,107.680732,56.153776,55.214721,59.238983,56.153776,56.153776,1.167348,1.167348,1.167348,-0.567135,-0.567135,-0.567135,1.734484,1.734484,1.734484,0.0,0.0,0.0,57.321124,142.282312,119.822524,137.840619,142.282312,...,0.0,0.0,0.0,138.818360,67.099528,64.130154,70.823853,67.099528,67.099528,0.283814,0.283814,0.283814,-0.752941,-0.752941,-0.752941,1.036755,1.036755,1.036755,0.0,0.0,0.0,67.383342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,103.509064,98.905969,114.225739,103.509064,103.509064,3.234103,3.234103,3.234103,-1.486107,-1.486107,-1.486107,4.720210,4.720210,4.720210,0.0,0.0,0.0,106.743167,56.209674,54.944488,59.034169,56.209674,56.209674,0.766928,0.766928,0.766928,-0.570675,-0.570675,-0.570675,1.337604,1.337604,1.337604,0.0,0.0,0.0,56.976602,142.184298,118.306859,137.537302,142.184298,...,0.0,0.0,0.0,137.742513,67.012450,63.815386,70.464099,67.012450,67.012450,-0.047166,-0.047166,-0.047166,-0.776005,-0.776005,-0.776005,0.728839,0.728839,0.728839,0.0,0.0,0.0,66.965283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,103.542467,97.961174,113.432244,103.542467,103.542467,2.027469,2.027469,2.027469,-1.734076,-1.734076,-1.734076,3.761545,3.761545,3.761545,0.0,0.0,0.0,105.569936,56.265572,54.788698,58.610212,56.265572,56.265572,0.377424,0.377424,0.377424,-0.551713,-0.551713,-0.551713,0.929138,0.929138,0.929138,0.0,0.0,0.0,56.642997,142.086283,118.233221,136.095045,142.086283,...,0.0,0.0,0.0,136.531757,66.925371,63.338272,69.760943,66.925371,66.925371,-0.404871,-0.404871,-0.404871,-0.808865,-0.808865,-0.808865,0.403995,0.403995,0.403995,0.0,0.0,0.0,66.520501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,103.575869,96.523252,112.649707,103.575869,103.575869,1.153681,1.153681,1.153681

In [18]:
predictions = final[['yhat_' + g for g in grouped.groups.keys()]]

In [19]:
predictions.reset_index(inplace=True)
predictions

,ds,yhat_11,yhat_12,yhat_14,yhat_21,yhat_22,yhat_23,yhat_24,yhat_31,yhat_32,yhat_33,yhat_34,yhat_41,yhat_42,yhat_43,yhat_44,yhat_51,yhat_52,yhat_54
0,2017-01-01,108.558315,57.766023,130.181685,188.589794,88.388462,327.729043,63.611645,253.019419,131.824274,NaN,88.234536,156.770233,91.661832,NaN,100.159068,139.406519,67.918359,NaN
1,2017-01-02,107.680732,57.321124,129.083750,186.611988,87.976488,322.358109,62.857913,251.878053,133.085166,820.421004,88.256868,154.988576,90.429969,NaN,99.679580,138.818360,67.383342,NaN
2,2017-01-03,106.743167,56.976602,128.233240,185.024696,87.416355,318.329631,62.401226,249.804281,131.039488,819.098410,87.631242,153.678706,89.991096,NaN,98.201121,137.742513,66.965283,NaN
3,2017-01-04,105.569936,56.642997,127.234272,183.224450,86.761149,312.524794,61.916434,247.312165,129.936025,804.292110,86.432464,152.218069,89.410858,NaN,97.351369,136.531757,66.520501,NaN
4,2017-01-05,104.729551,56.293220,126.466433,181.878527,86.386394,307.990141,61.291110,244.912381,128.267533,782.436481,85.404181,150.997894,89.062173,NaN,96.342967,135.409274,66.045691,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,2023-12-27,184.350047,55.341200,62.953650,203.903196,68.604542,208.643539,37.230276,229.832829,266.848890,447.311658,3.400092,182.986486,72.895269,188.083874,88.112124,203.310349,75.280938,131.047308
2552,2023-12-28,184.638640,55.131422,62.717259,203.931785,68.668268,205.627629,36.982362,228.990106,266.170841,429.054607,2.753402,183.049690,73.015298,187.703887,87.637665,203.649701,75.118630,130.834997
2553,2023-12-29,189.780432,56.748414,63.039202,208.823590,70.240513,206.269569,37.368731,234.631545,267.072218,422.135213,4.391786,192.166605,98.310247,189.702634,87.898604,208.633515,77.705158,145.655680
2554,2023-12-30,189.726425,56.471608,62.746949,208.363435,69.988362,204.939805,37.036605,234.581150,267.545666,411.432913,3.537702,191.851513,98.062705,189.077699,87.424908,208.680856,77.568985,145.577702


In [20]:
predictions =predictions.melt(['ds'], var_name='combined', value_name='value')
predictions

,ds,combined,value
0,2017-01-01,yhat_11,108.558315
1,2017-01-02,yhat_11,107.680732
2,2017-01-03,yhat_11,106.743167
3,2017-01-04,yhat_11,105.569936
4,2017-01-05,yhat_11,104.729551
...,...,...,...
46003,2023-12-27,yhat_54,131.047308
46004,2023-12-28,yhat_54,130.834997
46005,2023-12-29,yhat_54,145.655680
46006,2023-12-30,yhat_54,145.577702


In [21]:
predictions= predictions.replace('yhat_', '', regex=True)
predictions

,ds,combined,value
0,2017-01-01,11,108.558315
1,2017-01-02,11,107.680732
2,2017-01-03,11,106.743167
3,2017-01-04,11,105.569936
4,2017-01-05,11,104.729551
...,...,...,...
46003,2023-12-27,54,131.047308
46004,2023-12-28,54,130.834997
46005,2023-12-29,54,145.655680
46006,2023-12-30,54,145.577702


In [22]:
predictions['borough_id'] = predictions['combined'].str[0]
predictions['room_typeid']= predictions['combined'].str[-1:]

In [23]:
predictions = predictions[['ds','borough_id', 'room_typeid', 'value']]
predictions

,ds,borough_id,room_typeid,value
0,2017-01-01,1,1,108.558315
1,2017-01-02,1,1,107.680732
2,2017-01-03,1,1,106.743167
3,2017-01-04,1,1,105.569936
4,2017-01-05,1,1,104.729551
...,...,...,...,...
46003,2023-12-27,5,4,131.047308
46004,2023-12-28,5,4,130.834997
46005,2023-12-29,5,4,145.655680
46006,2023-12-30,5,4,145.577702


In [24]:
predictions = predictions.dropna()
predictions

,ds,borough_id,room_typeid,value
0,2017-01-01,1,1,108.558315
1,2017-01-02,1,1,107.680732
2,2017-01-03,1,1,106.743167
3,2017-01-04,1,1,105.569936
4,2017-01-05,1,1,104.729551
...,...,...,...,...
46003,2023-12-27,5,4,131.047308
46004,2023-12-28,5,4,130.834997
46005,2023-12-29,5,4,145.655680
46006,2023-12-30,5,4,145.577702


In [25]:
predictions.to_csv("predictions.csv")